<a href="https://colab.research.google.com/github/Waxpardo/PRA2031-Project/blob/Michail-Angelos/Monte%20Carlo%20Simulation%20for%20muons.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import random
import math

# Physical constants for muons

# Rest mass of a muon (kg)
MUON_MASS = 1.883531627e-28

# Mean lifetime of a muon at rest (seconds)
MUON_LIFETIME = 2.1969811e-6


# Muon class
class Muon:
    def __init__(self, x, y, vx, vy, radius=1e-15):
        # Position (meters)
        self.x = x
        self.y = y

        # Velocity components (m/s)
        self.vx = vx
        self.vy = vy

        # Physical properties
        self.mass = MUON_MASS
        self.radius = radius  # effective interaction radius

        # Muon state (alive or decayed)
        self.alive = True

    def move(self, dt):
        """
        Update muon position using simple kinematics:
        x_new = x + v * dt
        """
        self.x += self.vx * dt
        self.y += self.vy * dt

    def decay(self, dt):
        """
        Monte Carlo decay:
        Probability of decay in time dt is dt / tau
        """
        if random.random() < dt / MUON_LIFETIME:
            self.alive = False



# Collision system
class MuonCollisionSystem:
    def __init__(self, muons):
        # List of all muons in the system
        self.muons = muons

    def monte_carlo_collisions(self, trials):
        """
        Randomly sample muon pairs to check for collisions
        (Monte Carlo approach instead of O(N^2))
        """
        # Only consider muons that have not decayed
        alive_muons = [m for m in self.muons if m.alive]

        # Need at least two particles to collide
        if len(alive_muons) < 2:
            return

        # Perform random collision trials
        for _ in range(trials):
            m1, m2 = random.sample(alive_muons, 2)
            self.check_collision(m1, m2)

    def check_collision(self, m1, m2):
        """
        Check if two muons overlap (distance <= sum of radii)
        """
        dx = m2.x - m1.x
        dy = m2.y - m1.y
        distance = math.hypot(dx, dy)

        # Collision condition
        if distance <= m1.radius + m2.radius:
            self.elastic_collision(m1, m2)

    def elastic_collision(self, m1, m2):
        """
        Perform a 2D elastic collision using
        momentum conservation along the collision normal
        """
        dx = m2.x - m1.x
        dy = m2.y - m1.y
        distance = math.hypot(dx, dy)

        # Avoid division by zero
        if distance == 0:
            return

        # Unit normal vector
        nx = dx / distance
        ny = dy / distance

        # Relative velocity
        dvx = m1.vx - m2.vx
        dvy = m1.vy - m2.vy

        # Velocity component along the normal
        vn = dvx * nx + dvy * ny

        # If particles are moving apart, no collision response needed
        if vn > 0:
            return

        # Elastic collision impulse (classical)
        impulse = (2 * vn) / (m1.mass + m2.mass)

        # Update velocities according to conservation of momentum
        m1.vx -= impulse * m2.mass * nx
        m1.vy -= impulse * m2.mass * ny
        m2.vx += impulse * m1.mass * nx
        m2.vy += impulse * m1.mass * ny

    def update(self, dt, mc_trials):
        """
        Advance the simulation by one timestep:
        - Move muons
        - Apply decay
        - Perform Monte Carlo collision sampling
        """
        for m in self.muons:
            if m.alive:
                m.move(dt)
                m.decay(dt)

        self.monte_carlo_collisions(mc_trials)

